In [2]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from ta import add_all_ta_features
#########PARA FAANG STOCKS###########
def get_aapl_data(interval='1d', start_date='2011-01-01', end_date='2030-12-31'):
    try:
        aapl_data = yf.download('AAPL', start=start_date, end=end_date, interval=interval)
        return aapl_data
    except Exception as e:
        print(f"Error al obtener los datos: {e}")
        return None

def calculate_ibs(data):
    high_low_range = data["High"] - data["Low"]
    close_open_range = abs(data["Close"] - data["Low"])
    ibs = close_open_range / high_low_range
    return ibs

def calculate_ema(data, window=8):
    ema = data["Close"].ewm(span=window, adjust=False).mean()
    return ema

def backtest_strategy(data, initial_capital=10000):
    capital = initial_capital
    position = 0
    buy_price = 0
    gains_losses = []
    trades = []
    
    for i in range(len(data)):
        if data["Buy_Signal"].iloc[i]:
            # Buy signal
            if position == 0:
                buy_price = data["Close"].iloc[i]
                position = capital / buy_price
                capital = 0
                trades.append({"Type": "Buy", "Date": data.index[i], "Price": buy_price})
        elif data["Sell_Signal"].iloc[i]:
            # Sell signal
            if position > 0:
                sell_price = data["Close"].iloc[i]
                capital = position * sell_price
                position = 0
                gains_losses.append(capital - initial_capital)
                trades.append({"Type": "Sell", "Date": data.index[i], "Price": sell_price, "Gain/Loss": capital - initial_capital})
    
    # Calculate final gains and losses for open position
    if position > 0:
        final_sell_price = data["Close"].iloc[-1]
        capital = position * final_sell_price
        gains_losses.append(capital - initial_capital)
        trades.append({"Type": "Sell", "Date": data.index[-1], "Price": final_sell_price, "Gain/Loss": capital - initial_capital})
    
    total_gains_losses = sum(gains_losses)
    return total_gains_losses, trades

if __name__ == "__main__":
    start_date = '2011-01-01'
    end_date = '2030-12-31'
    interval = '1d'
    initial_capital = 10000

    data = get_aapl_data(interval=interval, start_date=start_date, end_date=end_date)

    if data is not None:
        # Calculate IBS and EMAs
        data = add_all_ta_features(data, open="Open", high="High", low="Low", close="Close", volume="Volume", fillna=True)
        data["IBS"] = calculate_ibs(data)
        data["EMA_8"] = calculate_ema(data, window=8)
        data["EMA_24"] = calculate_ema(data, window=24)

        # Create Buy and Sell signals
        data["Buy_Signal"] = (data["IBS"] <= 0.25) & (data["EMA_8"] < data["EMA_24"])
        data["Sell_Signal"] = (data["IBS"] >= 0.75) & (data["EMA_8"] > data["EMA_24"])

       # Backtest the strategy
    total_gains_losses, trades = backtest_strategy(data, initial_capital=initial_capital)

    # Print the total gains and losses
    print(f"Total Gains/Losses: {total_gains_losses:.2f} €")

    # Print the trade details in the daily journal
    print("\nDaily Journal (Trades):")
    for trade in trades:
        print(f"{trade['Type']} - Date: {trade['Date']} | Price: {trade['Price']} |", end=" ")
        if 'Gain/Loss' in trade:
            print(f"Gain/Loss: {trade['Gain/Loss']:.2f} €")
        else:
            print("Gain/Loss: N/A")

[*********************100%***********************]  1 of 1 completed


c:\Users\filip\AppData\Local\Programs\Python\Python311\Lib\site-packages\ta\trend.py:780: RuntimeWarning: invalid value encountered in scalar divide
  dip[idx] = 100 * (self._dip[idx] / value)
c:\Users\filip\AppData\Local\Programs\Python\Python311\Lib\site-packages\ta\trend.py:785: RuntimeWarning: invalid value encountered in scalar divide
  din[idx] = 100 * (self._din[idx] / value)


Total Gains/Losses: 108539.88 €

Daily Journal (Trades):
Buy - Date: 2011-03-16 00:00:00 | Price: 11.786070823669434 | Gain/Loss: N/A
Sell - Date: 2011-03-29 00:00:00 | Price: 12.534285545349121 | Gain/Loss: 126.97 €
Buy - Date: 2011-04-06 00:00:00 | Price: 12.072856903076172 | Gain/Loss: N/A
Sell - Date: 2011-04-25 00:00:00 | Price: 12.607500076293945 | Gain/Loss: 221.16 €
Buy - Date: 2011-05-16 00:00:00 | Price: 11.903571128845215 | Gain/Loss: N/A
Sell - Date: 2011-07-01 00:00:00 | Price: 12.259285926818848 | Gain/Loss: 287.53 €
Buy - Date: 2011-08-19 00:00:00 | Price: 12.715356826782227 | Gain/Loss: N/A
Sell - Date: 2011-08-26 00:00:00 | Price: 13.699286460876465 | Gain/Loss: 464.55 €
Buy - Date: 2011-10-03 00:00:00 | Price: 13.378570556640625 | Gain/Loss: N/A
Sell - Date: 2011-10-13 00:00:00 | Price: 14.586786270141602 | Gain/Loss: 687.12 €
Buy - Date: 2011-11-10 00:00:00 | Price: 13.757857322692871 | Gain/Loss: N/A
Sell - Date: 2011-12-09 00:00:00 | Price: 14.057856559753418 | Gai